<a href="https://colab.research.google.com/github/Sajeeb07/CricketBot/blob/main/CricketBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation**

In [1]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 39.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.5 MB/s eta 0:00:00
     ━━

**Import Libraries**




In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [3]:
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown



def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
# Used to securely store your API key
from google.colab import userdata

In [5]:
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = userdata.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HUGGINGFACEHUB_API_TOKEN"

**Import Document**

In [6]:
loader = PyPDFDirectoryLoader("/content/sample_data/Data")
docs = loader.load()

In [7]:
docs

[Document(page_content=' \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nTHE ROYAL  MARINES  \n \n \n \nCricket  \n', metadata={'source': '/content/sample_data/Data/cricketdata.pdf', 'page': 0}),
 Document(page_content=' \n  \n \n \n \n \n \n \n \n \n \n \n \nCONTENTS  \n \nStudent  Notes  Teacher  Notes  \n \n \nPrepared  for The Royal  Navy  in Association  with The National  Cricket  \nAssociation  and produced  by Education  & Youth  Ltd, London.  \nIf there is any support you feel The Royal Navy can give regarding this  \nproject  please  contact  0870  333 0423.  \n \n \nAcknowledgements  \nThese  notes  have been  compiled  by Bob Carter  – National  Coach,  \n(Coach  Education)  The National  Cricket  Association  \nTechnical  Editor:  P Edwards  M.A. MESSAGE  FROM  THE NATIONAL  CRICKET  ASSOCIATION  3 \nINTRODUCTION  TO THE MODULE  3 \nSKILLS  DEVELOPMENT  SHEET  4 \nUNIT  1 INTRODUCTION  TO CRICKET  MODULE  5 \nUNIT  2 TECHNIQUES  AND SKILLS  6 \n 1 FIELDING  6 \n 2 BOWLING  

**Splitting Test and Chunking**

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [9]:
len(chunks)

226

In [10]:
chunks[0]

Document(page_content='THE ROYAL  MARINES  \n \n \n \nCricket', metadata={'source': '/content/sample_data/Data/cricketdata.pdf', 'page': 0})

In [11]:
chunks[1]

Document(page_content='CONTENTS  \n \nStudent  Notes  Teacher  Notes  \n \n \nPrepared  for The Royal  Navy  in Association  with The National  Cricket  \nAssociation  and produced  by Education  & Youth  Ltd, London.  \nIf there is any support you feel The Royal Navy can give regarding this', metadata={'source': '/content/sample_data/Data/cricketdata.pdf', 'page': 1})

In [12]:
chunks[2]

Document(page_content='project  please  contact  0870  333 0423.  \n \n \nAcknowledgements  \nThese  notes  have been  compiled  by Bob Carter  – National  Coach,  \n(Coach  Education)  The National  Cricket  Association  \nTechnical  Editor:  P Edwards  M.A. MESSAGE  FROM  THE NATIONAL  CRICKET  ASSOCIATION  3', metadata={'source': '/content/sample_data/Data/cricketdata.pdf', 'page': 1})

In [13]:
chunks[3]

Document(page_content='INTRODUCTION  TO THE MODULE  3 \nSKILLS  DEVELOPMENT  SHEET  4 \nUNIT  1 INTRODUCTION  TO CRICKET  MODULE  5 \nUNIT  2 TECHNIQUES  AND SKILLS  6 \n 1 FIELDING  6 \n 2 BOWLING  9 \n 3 BATTING  14 \n 4 WICKET  KEEPING  22 \nUNIT  3 TACTICS  AND FIELD  PLACEMENT  25 \nUNIT  4 UMPIRING  28', metadata={'source': '/content/sample_data/Data/cricketdata.pdf', 'page': 1})

In [14]:
chunks[4]

Document(page_content='UNIT  4 UMPIRING  28 \nUNIT  5 PHYSICAL  CONDITIONING  FOR CRICKET  30 \nUNIT  6 HISTORY  OF CRICKET  34 \nUNIT  7 THE COMPETITIVE  STRUCTURE  OF THE SPORT.  35 \nUNIT  8 USEFUL  INFORMATION  36', metadata={'source': '/content/sample_data/Data/cricketdata.pdf', 'page': 1})

**Embedding**

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Vector Store**

In [16]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [17]:
vectorstore

In [18]:
query = "there are may different levels of the game ranging from what?"
search = vectorstore.similarity_search(query)

In [19]:
to_markdown(search[0].page_content)

> which  teachers  and coaches  should endeavour  to foster at  all levels.  
> There are may different levels of the game ranging from the Five Day  
> Test Match,  the afternoon  club match,  the indoor  six a side adults,  
> Kwik Cricket, Cricket Pairs game played by 10 year olds to the

**Retriever**

In [20]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [21]:
retriever.get_relevant_documents(query)

[Document(page_content='which  teachers  and coaches  should endeavour  to foster at  all levels.  \nThere are may different levels of the game ranging from the Five Day  \nTest Match,  the afternoon  club match,  the indoor  six a side adults,  \nKwik Cricket, Cricket Pairs game played by 10 year olds to the', metadata={'page': 4, 'source': '/content/sample_data/Data/cricketdata.pdf'}),
 Document(page_content='UNIT  5 \nPhysical  Conditioning  for Cricket  \nThe degree  of fitness  required  to play depends  upon  the level and type \nof match  played.  Generally  cricket  is played  over longer  periods  of \ntime than other  games;  therefore  the training  has different  aspects.  To', metadata={'page': 29, 'source': '/content/sample_data/Data/cricketdata.pdf'}),
 Document(page_content='UNIT  2 \nTechniques  and Skills  \nIn order  to play cricket  to examination  level,  students  should  have a \ngood  practical  and theoretical  grasp  of the 4 major  activities  of the \ngame: 

In [22]:
#connect to google drive
from google.colab import drive

In [23]:
drive.mount('/content/drive')

Mounted at /content/drive


**LLM**

In [24]:
llm = LlamaCpp(
    model_path= "/content/drive/MyDrive/Chatbotmodel/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/Chatbotmodel/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.a

**RAG Chain**

In [25]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [26]:
template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [27]:
prompt = ChatPromptTemplate.from_template(template)

In [28]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
response = rag_chain.invoke("what is long throw?")


llama_print_timings:        load time =    4290.21 ms
llama_print_timings:      sample time =      52.33 ms /    77 runs   (    0.68 ms per token,  1471.32 tokens per second)
llama_print_timings: prompt eval time =   28904.47 ms /    55 tokens (  525.54 ms per token,     1.90 tokens per second)
llama_print_timings:        eval time =   51733.35 ms /    76 runs   (  680.70 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =   81058.90 ms /   131 tokens


In [30]:
to_markdown(response)

> Long throw refers to a type of throw in which the ball is thrown as far as possible, usually by an athlete such as a javelin thrower or discus thrower. It requires a combination of strength, technique, and precision to achieve the desired distance. In sports, long throws are often used as a way to score points or gain an advantage over opponents.

In [31]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ",result)

Input Prompt: how should you position your hands and adjust your head when catching a flat trajectory ball aimed at head height?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4290.21 ms
llama_print_timings:      sample time =      32.30 ms /    49 runs   (    0.66 ms per token,  1517.26 tokens per second)
llama_print_timings: prompt eval time =   16477.96 ms /    32 tokens (  514.94 ms per token,     1.94 tokens per second)
llama_print_timings:        eval time =   32415.87 ms /    48 runs   (  675.33 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   49153.69 ms /    80 tokens


Answer:  The best way to catch a ball is to stand with your feet shoulder width apart and to move your hands in the direction of the ball movement. Adjusting the head is not necessary as it is not directly involved in the catching process.
Input Prompt: what are some general points for fielders to keep in mind during a cricket match?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4290.21 ms
llama_print_timings:      sample time =     136.21 ms /   200 runs   (    0.68 ms per token,  1468.31 tokens per second)
llama_print_timings: prompt eval time =   13789.85 ms /    27 tokens (  510.74 ms per token,     1.96 tokens per second)
llama_print_timings:        eval time =  136516.43 ms /   199 runs   (  686.01 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =  151402.13 ms /   226 tokens


Answer:   Some general points for fielders to keep in mind during a cricket match are: Always be alert and aware of the ball. Try to anticipate the direction of the ball and the batsman's shot. Communicate with your teammates about the ball direction, the batsman, and your position. Stay focused on the game and avoid getting distracted. Stay alert at all times, even when the ball is not coming towards you. Keep your eyes on the ball as much as possible. Try to anticipate where the ball is going to go and where the batsman is going to hit it. Stay focused on the game and avoid getting distracted. Always be prepared to move quickly and change direction. Try to anticipate where the ball is going to go and where the batsman is going to hit it. Communicate with your teammates about the ball direction, the batsman, and your position. Stay focused on the game and avoid getting distracted.
Input Prompt: when were the modern laws of cricket first written down and printed?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4290.21 ms
llama_print_timings:      sample time =     103.88 ms /   156 runs   (    0.67 ms per token,  1501.67 tokens per second)
llama_print_timings: prompt eval time =   11054.21 ms /    23 tokens (  480.62 ms per token,     2.08 tokens per second)
llama_print_timings:        eval time =  107889.23 ms /   155 runs   (  696.06 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =  119776.94 ms /   178 tokens


Answer:  The modern Laws of Cricket were first written down in 1844 by the Marylebone Cricket Society (MCC) in England, and it was published in 1845. The Laws have been revised several times since then, most recently in 2014, to reflect changes in the game over time. The Laws are designed to ensure fair play, protect players, and promote the spirit of cricket. They cover areas such as the equipment used, the pitch, the bowler, the batsman, the wicketkeeper, the umpire, and the captain. Any changes to the Laws must be approved by the MCC, which is responsible for overseeing the game of cricket worldwide.
Input Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
